In [3]:
# !pip install cerebras_cloud_sdk

In [72]:
import ast
import os
import re

from tqdm import tqdm

import pandas as pd
from cerebras.cloud.sdk import Cerebras

client = Cerebras(
        api_key = os.environ.get("CEREBRAS_API_KEY")
)


In [3]:

stream = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Who are you?"
        }
    ],
    model="llama-4-scout-17b-16e-instruct",
    stream=True,
    max_completion_tokens=2048,
    temperature=0.2,
    top_p=1
)

for chunk in stream:
  print(chunk.choices[0].delta.content or "", end="")

I'm an AI assistant designed by Meta. I'm here to answer your questions, share interesting ideas and maybe even surprise you with a fresh perspective. What's on your mind?

In [4]:
system_prompt = "Solve the following question with highly detailed step by step explanation. Write the correct answer inside a dictionary at the end in the following format. The key 'answer' has a list which can be filled by all correct options or by a number as required while answering the question. For example for question with correct answer as option (a), return {'answer':[a]} at the end of solution. For question with multiple options'a,c' as answers, return {'answer':[a,c]}. And for question with numerical values as answer (say 1.33), return {'answer':[1.33]}"

In [6]:
df = pd.read_excel('/Users/mohdzaki/Documents/github/MaSTeA/all_QA.xlsx')

In [24]:
questions = list(df.QUESTION.apply(lambda x: x[2:-2]))

In [40]:
answers = []
for q in tqdm(questions):
    question = q.strip()
    stream = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": q
        },
    ],
    model="llama-4-scout-17b-16e-instruct",
    stream=False,
    max_completion_tokens=2048,
    temperature=0.,
    top_p=1
    )
    answers.append(stream)
    # break

100%|███████████████████████████████████████████████████████████████████| 650/650 [22:47<00:00,  2.10s/it]


In [77]:
extraction = []
for ans in tqdm(answers):
    try:
        idx = ans.to_dict()['choices'][0]['message']['content'].index("{'answer")
        extraction.append(ans.to_dict()['choices'][0]['message']['content'][idx:])
    except:
        
        extraction.append(ans)
        print(len(extraction))

100%|████████████████████████████████████████████████████████████████| 650/650 [00:00<00:00, 42081.59it/s]

110
557


In [78]:
df['LLAMA-SCOUT_answers_complete'] = answers

In [175]:
eval_prompt = 'You are an evaluation expert. You will be provided with correct answer and the answer to be evaluated. Please return 1 if both are correct, else return 0. DO NOT OUTPUT ANYTHING ELSE.'

In [176]:
from tqdm import tqdm, trange

In [181]:
import time

In [182]:
len(score)

31

'1'

In [185]:
score = []
for i in trange(len(df['LLAMA-SCOUT'])):
    correct_answer = df['Correct Answer'].iloc[i]
    evaluate = df['LLAMA-SCOUT'].iloc[i]
    msg = f'Correct answer : {correct_answer}. Evaluate : {evaluate}'
    
    stream = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": eval_prompt
        },
        {
            "role": "user",
            "content": msg
        },
    ],
    model="llama-4-scout-17b-16e-instruct",
    stream=False,
    max_completion_tokens=5,
    temperature=0.,
    top_p=1
    )
    score.append(stream)
    time.sleep(1)
    # break

100%|███████████████████████████████████████████████████████████████████| 650/650 [23:36<00:00,  2.18s/it]


In [189]:
df['LLAMA-SCOUT-score'] = score

In [196]:
nums = []
for x in score:
    try:
        nums.append(int(x.to_dict()['choices'][0]['message']['content']))
    except:
        nums.append('check me')
        print(x.to_dict()['choices'][0]['message']['content'])

## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
To be evaluated: The
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:
## Step 1:


In [197]:
len(nums)

650

In [198]:
df['nums'] = nums

In [199]:
df.to_excel('cerebras.xlsx',index=False)

In [191]:
df['scr'] = df['LLAMA-SCOUT-score'].apply(lambda x: int(x.to_dict()['choices'][0]['message']['content']))

ValueError: invalid literal for int() with base 10: '## Step 1:'

In [179]:
stream.to_dict()['choices'][0]['message']['content']

{'id': 'chatcmpl-0bf48d85-9526-46b2-a5b4-59c70f4d9ab3',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant', 'content': '1'}}],
 'created': 1744491169,
 'model': 'llama-4-scout-17b-16e-instruct',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_019831f7fe7da682d517',
 'time_info': {'completion_time': 0.00265898,
  'prompt_time': 0.001828023,
  'queue_time': 0.00011043,
  'total_time': 0.020841360092163086,
  'created': 1744491169},
 'usage': {'completion_tokens': 2, 'prompt_tokens': 69, 'total_tokens': 71}}

In [88]:
extraction[109] = "{'answer': ['B']}"

In [90]:
extraction[556] = "{'answer': ['A']}"

In [91]:
df['LLAMA-SCOUT'] = extraction

In [106]:

k

"{'answer':[B]}.\n\nThe final answer is: \n{'answer':['B']}"

In [117]:
df['LLAMA-SCOUT'].iloc[518] = "{'answer':2.42 * 10^{12}}"

/var/folders/h0/vgvyg41d3dv4zc8f9sy78yb40000gn/T/ipykernel_1567/1718070237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LLAMA-SCOUT'].iloc[518] = "{'answer':2.42 * 10^{12}}"


In [118]:
df['LLAMA-SCOUT'].iloc[518]

"{'answer':2.42 * 10^{12}}"

In [126]:
ee

"{'answer':2.42 * 10^{12}"

In [129]:
dictout = len(df)*['0000']
for i, k in enumerate(df['LLAMA-SCOUT']):
    k = k.split('}')[0]
    k += '}'
    k = k.replace("\\",'')
    try:
        # print(i)
        dictout[i] = ast.literal_eval(k)
    except:
        ee = k
        ee = ee.replace('[',"'[")
        ee = ee.replace(']',"]'")
        if i == 518:
            ee = "{'answer':'[2.42 * 10**12]'}"
        dictout[i] = ast.literal_eval(ee)
        # try:
        #     print(i)
        # except:
        #     print(i)
        # try:
        #     k = k.split('}')[0]
        #     k += '}'
        #     ee = k
        #     ee = ee.replace('[',"'[")
        #     ee = ee.replace(']',"]'")
        #     dictout.append(ast.literal_eval(ee))
        # except:
        #     print(i)
            
        # print(i, k)

In [130]:
len(dictout)

650

In [131]:
df['LLAMA-SCOUT-filter'] = dictout

In [136]:
df['LLAMA-SCOUT-filter'] = df['LLAMA-SCOUT-filter'].apply(lambda x: type(x['answer']))

In [152]:
def normalize(x):
    try:
        if type(x) == list:
            if len(x)==1:
                return x[0]
            else:
                return x
        elif type(x) == str:
            return ast.literal_eval(x)
        elif type(x) == float:
            return x
    except:
        return x

In [155]:
df['LLAMA-SCOUT-filter'] = df['LLAMA-SCOUT-filter'].apply(lambda x: normalize(x))

In [156]:
df['LLAMA3-8B']

0         D
1         C
2         A
3         B
4         A
       ... 
645     396
646     200
647       0
648    3.67
649      20
Name: LLAMA3-8B, Length: 650, dtype: object

In [ ]:
for i in

In [159]:
df['LLAMA-SCOUT-filter'].apply(lambda x: ast.literal_eval(x))

ValueError: malformed node or string: <ast.Name object at 0x30c50ba90>

In [132]:
# df['LLAMA-SCOUT'] = df['LLAMA-SCOUT'].apply(lambda x: ast.literal_eval(x)['answer'])

In [137]:
def extract_corresponding_value(question, correct_answer):
    # Find the option corresponding to the correct answer letter
    match = re.search(r'\({}\)\s*(.*?)(?:\s*\(|$)'.format(correct_answer), question)
    if match:
        extracted_value = match.group(1).strip()
        if correct_answer == 'D':
            extracted_value = extracted_value[:-2]  # Exclude the last two characters
        return extracted_value
    return None


# Function to check if a prediction is correct for a given row
def is_correct_prediction(row):
    ground_truth = str(row['Correct Answer'])
    # Change the below line with the model name that you are interested in evaluating
    predicted = row['LLAMA-SCOUT-filter'] #row['LLAMA3-8B']
    question_type = row['Question Type']
    
    # Select NUM category
    if question_type == 'NUM':
        if 'to' in ground_truth or ':' in ground_truth:
            try:
                range_numbers = re.findall(r'\d+\.\d+|\d+', ground_truth)
                range_start, range_end = map(float, [range_numbers[0], range_numbers[-1]])
                if range_start <= float(predicted) <= range_end:
                    return True
            except (ValueError, IndexError):
                pass
        elif str(predicted) == ground_truth:
            return True

    # For MATCH category
    else:
        # First, check if the predicted answer matches the ground truth
        if str(predicted) == ground_truth:
            return True
        # If not, check if the ground truth matches the corresponding answer
        corresponding_answer = row['Corresponding Value']
        if corresponding_answer and predicted == corresponding_answer:
            return True

    # For other question types, check for exact match
    return str(predicted) == ground_truth


def calculate_accuracy(df):
    results_by_category = df.apply(is_correct_prediction, axis=1).groupby([df['Question Type'], df['TOPIC']]).agg(['sum', 'count'])

    accuracies = {}
    total_correct_predictions = 0
    total_questions = 0

    print("\nResults by category (Correct Predictions / Total Questions):")
    for category, result in results_by_category.iterrows():
        correct_predictions = result['sum']
        total = result['count']
        accuracy = correct_predictions / total
        accuracies[category] = accuracy
        total_correct_predictions += correct_predictions
        total_questions += total
        print(f"{category}: {correct_predictions} / {total} (Accuracy: {accuracy:.2%})")

    # Calculate accuracy for specific question types (NUM, MATCH, etc.)
    for question_type in df['Question Type'].unique():
        if question_type not in accuracies:
            accuracies[question_type] = 0
        questions_subset = df[df['Question Type'] == question_type]
        correct_predictions_subset = questions_subset.apply(is_correct_prediction, axis=1).sum()
        total_questions_subset = len(questions_subset)
        accuracy_subset = correct_predictions_subset / total_questions_subset
        print(f"{question_type} Accuracy: {correct_predictions_subset} / {total_questions_subset} (Accuracy: {accuracy_subset:.2%})")

    overall_accuracy = total_correct_predictions / total_questions
    print(f"\nOverall Accuracy: {total_correct_predictions} / {total_questions} (Accuracy: {overall_accuracy:.2%})")

    return accuracies, overall_accuracy

category_accuracies, overall_accuracy = calculate_accuracy(df)

TypeError: float() argument must be a string or a number, not 'list'